<a href="https://colab.research.google.com/github/islombek-cs/AI-Powered-Quiz-App/blob/main/RobustaVisionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/islombek-cs/RobustaVision.git
%cd RobustaVision

Cloning into 'RobustaVision'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 30 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 10.67 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/RobustaVision


In [ ]:
!pip install torch torchvision matplotlib numpy tqdm

In [ ]:
!mkdir -p src

In [2]:
# Creating the training script
%%writefile src/train_baseline.py
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

def get_loaders(batch_size=128):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616)),
    ])

    train_ds = datasets.CIFAR10(root="./data", train=True,
                                download=True, transform=transform_train)
    test_ds  = datasets.CIFAR10(root="./data", train=False,
                                download=True, transform=transform_test)

    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2),
        DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2),
    )

def main(epochs=5, lr=1e-3, device="cuda"):
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    train_loader, test_loader = get_loaders()

    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, epochs + 1):
        model.train()
        correct, total, loss_sum = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item() * x.size(0)
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)

        print(f"Epoch {epoch}: loss={loss_sum/total:.4f}, "
              f"train_acc={correct/total:.3f}")

    torch.save(model.state_dict(), "resnet18_cifar10_baseline.pth")

if __name__ == "__main__":
    main()


Writing src/train_baseline.py


FileNotFoundError: [Errno 2] No such file or directory: 'src/train_baseline.py'

In [1]:
!python src/train_baseline.py

python3: can't open file '/content/src/train_baseline.py': [Errno 2] No such file or directory
